In [5]:
from src.robinhood_api.api_access import CryptoAPITrading
import pandas as pd

%load_ext autoreload
%autoreload 2

2025-03-10 19:13:51,665 - INFO - NumExpr defaulting to 8 threads.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# Create an instance of the CryptoAPITrading which accesses the Robinhood API
api_trading_client = CryptoAPITrading()

api_trading_client.account.get_account()

{'account_number': '311061556262',
 'status': 'active',
 'buying_power': '1.4400',
 'buying_power_currency': 'USD'}

In [27]:
api_trading_client.account.get_holdings()

{'next': None,
 'previous': None,
 'results': [{'account_number': '311061556262',
   'asset_code': 'PEPE',
   'total_quantity': '115412.000000000000000000',
   'quantity_available_for_trading': '115412.000000000000000000'},
  {'account_number': '311061556262',
   'asset_code': 'AVAX',
   'total_quantity': '0.019000000000000000',
   'quantity_available_for_trading': '0.019000000000000000'}]}

In [28]:
# ask_inclusive_of_buy_spread is the market buy
# price is the price that robinhood shows

# want to make a chart that updates every 10 seconds wit the price

bid, ask = api_trading_client.market_data.get_estimated_price(symbol="BTC-USD", side="both", quantity=1.0)['results']

In [12]:
import os
import glob

def load_data(ticker, interval, start_date=None, end_date=None, project_root='/Users/nathanielislas/PycharmProjects/CryptoTrading/'):
    # Get project root path
    #project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
    
    # Define data path relative to project root
    path_pattern = os.path.join(project_root, 'data', ticker, interval, '*.parquet')
    files = glob.glob(path_pattern)

    df_list = []

    for file in files:
        date = os.path.basename(file).split('.')[0]

        # Load file if within date range or if no date range is specified
        if (start_date is None or end_date is None) or (start_date <= date <= end_date):
            df = pd.read_parquet(file)
            df_list.append(df)

    # Check if any data was loaded
    if not df_list:
        print(f"No data found for {ticker} in the specified date range.")
        return pd.DataFrame()  # Return empty DataFrame if no data

    # Concatenate and return combined DataFrame
    return pd.concat(df_list).reset_index(drop=True)

In [13]:
data = load_data(ticker='BTC-USD', interval='1m')
data

,symbol,price_bid,quantity,side_bid,bid_inclusive_of_sell_spread,sell_spread,timestamp,price_ask,side_ask,ask_inclusive_of_buy_spread,buy_spread,date
0,BTC-USD,78444.101518,1,bid,78442.3973,0.00002173,2025-03-10 18:20:12.133259754+00:00,78444.101518,ask,78445.805736,0.00002173,2025-03-10
1,BTC-USD,78542.3358165,1,bid,78532.2312,0.00012865,2025-03-10 18:21:12.416935357+00:00,78542.3358165,ask,78552.440433,0.00012865,2025-03-10
2,BTC-USD,78421.529816,1,bid,78418.8476,0.0000342,2025-03-10 18:22:12.672952476+00:00,78421.529816,ask,78424.212032,0.0000342,2025-03-10
3,BTC-USD,78427.363638,1,bid,78429.7697,0.00003068,2025-03-10 18:23:13.032284341+00:00,78427.363638,ask,78424.957576,0.00003068,2025-03-10
4,BTC-USD,78352.334912,1,bid,78352.4104,0.00000096,2025-03-10 18:24:13.283903627+00:00,78352.334912,ask,78352.259424,0.00000096,2025-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...
244,BTC-USD,79649.583755,1,bid,79647.4871,0.00002632,2025-03-10 22:55:50.064633444+00:00,79649.583755,ask,79651.68041,0.00002632,2025-03-10
245,BTC-USD,79596.251392,1,bid,79592.4605,0.00004763,2025-03-10 22:56:50.333976251+00:00,79596.251392,ask,79600.042284,0.00004763,2025-03-10
246,BTC-USD,79549.9234975,1,bid,79551.9399,0.00002535,2025-03-10 22:57:50.589582660+00:00,79549.9234975,ask,79547.907095,0.00002535,2025-03-10
247,BTC-USD,79502.658382,1,bid,79505.5938,0.00003692,2025-03-10 22:58:50.870716581+00:00,79502.658382,ask,79499.722964,0.00003692,2025-03-10


In [14]:
# Load the Parquet file
file_path = '/Users/nathanielislas/PycharmProjects/CryptoTrading/data/BTC-USD/1m/2025-03-10.parquet'
df = pd.read_parquet(file_path)

# Get the size of the Parquet file in KB
parquet_size = os.path.getsize(file_path) / 1024  # Convert bytes to KB

# Save as CSV
csv_file_path = '/Users/nathanielislas/PycharmProjects/CryptoTrading/data/BTC-USD/1m/2025-03-10.csv'
df.to_csv(csv_file_path, index=False)

# Get the size of the CSV file in KB
csv_size = os.path.getsize(csv_file_path) / 1024  # Convert bytes to KB

# Print the results
print(f"Parquet File Size: {parquet_size:.2f} KB")
print(f"CSV File Size: {csv_size:.2f} KB")
print(f"CSV is approximately {csv_size / parquet_size:.2f} times larger than Parquet.")

Parquet File Size: 19.35 KB
CSV File Size: 33.58 KB
CSV is approximately 1.74 times larger than Parquet.


In [18]:
# Define file paths
parquet_file = '/Users/nathanielislas/PycharmProjects/CryptoTrading/data/BTC-USD/1m/2025-03-10.parquet'
csv_file = '/Users/nathanielislas/PycharmProjects/CryptoTrading/data/BTC-USD/1m/2025-03-10.csv'

# Define the number of iterations
iterations = 1000
parquet_times = []
csv_times = []

# Measure time to read Parquet file multiple times
for _ in range(iterations):
    start_time = time.time()
    pd.read_parquet(parquet_file)
    parquet_times.append(time.time() - start_time)

# Measure time to read CSV file multiple times
for _ in range(iterations):
    start_time = time.time()
    pd.read_csv(csv_file)
    csv_times.append(time.time() - start_time)

# Calculate average read times
avg_parquet_time = sum(parquet_times) / iterations
avg_csv_time = sum(csv_times) / iterations

# Print results
print(f"Average Parquet Read Time: {avg_parquet_time:.4f} seconds")
print(f"Average CSV Read Time: {avg_csv_time:.4f} seconds")
print(f"CSV is approximately {avg_csv_time / avg_parquet_time:.2f} times slower than Parquet.")

Average Parquet Read Time: 0.0034 seconds
Average CSV Read Time: 0.0012 seconds
CSV is approximately 0.36 times slower than Parquet.
